In [ ]:
import requests
import os
from datetime import datetime
from pathlib import Path
import pyarrow
import geopandas as gpd 
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import contextily as cx
import seaborn as sns
import matplotlib
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import csv

def strip_axes(ax):
    """Despine axis and remove ticks and labels."""
    sns.despine(ax=ax, left=True, bottom=True)
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

root_dir = Path("~/Desktop/Desktop/epidemiology_PhD/00_repos/").expanduser()
# root_dir = "/Volumes/squirrel-utopia 1/los_angeles_2025_fire_disasters_exp/"

crs = 26911

In [ ]:
# Ready to read data
data_dir = root_dir / "los_angeles_2025_fire_disasters_exp/data"

sep_fires_path =  data_dir / "sep_fires.parquet"
kaiser_zctas = data_dir / "kp_exp_zctas_2025_01_16.csv"
uw_zctas = data_dir / "zcta_exposures_2025_01_16.csv"
zctas_path = data_dir / "tl_2020_us_zcta520.shp"
# zctas = gpd.read_parquet(zctas_path).assign(zcta = lambda x: x.zcta.astype(int)).to_crs("EPSG:3857")

fires = gpd.read_parquet(sep_fires_path).to_crs(epsg=crs)
kaiser_exp = pd.read_csv(kaiser_zctas)
uw_exp = pd.read_csv(uw_zctas)
zctas = gpd.read_file(zctas_path).to_crs(epsg=crs)
zctas = zctas[['geometry', 'ZCTA5CE20']].rename(columns={'ZCTA5CE20': 'zcta'})

In [ ]:
def create_exposure_subplot(kaiser_exp, uw_exp, zctas_orig, fires, buffer_size, ax):
    """
    Create exposure plot for a specific buffer size on a given axis
    
    Parameters:
    -----------
    kaiser_exp : DataFrame
        Kaiser exposure data
    uw_exp : DataFrame
        UW exposure data
    zctas_orig : GeoDataFrame
        Original ZCTA shapes
    fires : GeoDataFrame
        Fire boundary data
    buffer_size : str
        Buffer size (e.g., '0.5', '1', '10', '20')
    ax : matplotlib.axes.Axes
        Axis to plot on
    """
    exposed_col = f'exposed_{buffer_size}buffer'
    kaiser_zctas = set(kaiser_exp['zcta'].unique())
    uw_zctas = set(uw_exp[uw_exp[exposed_col] == 1]['zcta'].unique())
    
    # overlapping ZCTAs
    overlapping_zctas = kaiser_zctas.intersection(uw_zctas)
    only_kaiser = kaiser_zctas - uw_zctas
    only_uw = uw_zctas - kaiser_zctas
    all_zctas = kaiser_zctas.union(uw_zctas)
    
    all_zctas = set(str(x) for x in all_zctas)
    overlapping_zctas = set(str(x) for x in overlapping_zctas)
    only_kaiser = set(str(x) for x in only_kaiser)
    only_uw = set(str(x) for x in only_uw)

    # Create CSV for this buffer size
    with open(f'zcta_categories_{buffer_size}km.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['zcta', 'overlapping', 'only_kaiser', 'only_uw'])
        
        for zcta in sorted(all_zctas):
            writer.writerow([
                zcta,
                1 if zcta in overlapping_zctas else 0,
                1 if zcta in only_kaiser else 0,
                1 if zcta in only_uw else 0
            ])
    
    # subset and copy ZCTA shapes
    zctas = zctas_orig[zctas_orig['zcta'].isin(all_zctas)].copy()
    
    # category column
    zctas.loc[zctas['zcta'].isin(overlapping_zctas), 'category'] = 'Both'
    zctas.loc[zctas['zcta'].isin(only_kaiser), 'category'] = 'Only Kaiser'
    zctas.loc[zctas['zcta'].isin(only_uw), 'category'] = 'Only UW'
    
    # convert geometries to EPSG:3857 to match base map 
    zctas = zctas.to_crs("EPSG:3857")
    fires = fires.to_crs("EPSG:3857")
    
    colors = ['#800080', '#FF0000', '#0066FF']  # purple, red, blue
    zctas.plot(
        column='category',
        ax=ax,
        legend=False,  # We'll create a single legend for the entire figure
        alpha=0.4,
        categorical=True,
        edgecolor="lightgrey",
        cmap=matplotlib.colors.ListedColormap(colors)
    )
    fires.boundary.plot(ax=ax, color="#8A3324", linewidth=1)
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
    ax.set_title(f"{buffer_size}km buffer", pad=10)
    strip_axes(ax)
    for idx, row in zctas.iterrows():
        centroid = row.geometry.centroid
        ax.text(
            centroid.x,
            centroid.y,
            str(row["zcta"]),
            fontsize=7
        )
    
    return zctas  

buffer_sizes = ['0.5', '1', '10', '20']
date_str = "14 January 2025"
fig = plt.figure(figsize=(20, 14))
fig.suptitle(f"Exposed ZCTAs by Buffer Size\n{date_str}", fontsize=16, y=0.95)

for i, buffer_size in enumerate(buffer_sizes):
    ax = plt.subplot(2, 2, i+1)
    zctas = create_exposure_subplot(
        kaiser_exp,
        uw_exp,
        zctas,
        fires,
        buffer_size,
        ax
    )
legend_elements = [
    Patch(facecolor='#800080', alpha=0.4, label='Both', edgecolor='lightgrey'),
    Patch(facecolor='#FF0000', alpha=0.4, label='Only Kaiser', edgecolor='lightgrey'),
    Patch(facecolor='#0066FF', alpha=0.4, label='Only UW', edgecolor='lightgrey'),
    Line2D([0], [0], color='black', linewidth=0.7, label='Fire Boundary')
]
fig.legend(
    handles=legend_elements,
    loc='center',
    bbox_to_anchor=(0.5, 0.02),
    ncol=4,
    fontsize=12
)

plt.tight_layout()